In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import pickle as pkl
import tsfresh as tsf
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute

#Classification with KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
#Classification with KNN
#Function to label the data
def getData(data, label):
    data['label'] = label
    return data

#Label Data
#Label Data Walk
df_walk1 = getData(df_walk1, 'walk')
df_walk2 = getData(df_walk2, 'walk')
df_walk3 = getData(df_walk3, 'walk')
df_walk4 = getData(df_walk4, 'walk')


#Label Data PushUps
df_push1 = getData(df_push1, 'pushups')
df_push2 = getData(df_push2, 'pushups')
df_push3 = getData(df_push3, 'pushups')
df_push4 = getData(df_push4, 'pushups')
df_push5 = getData(df_push5, 'pushups')

#Label Data JumpingJacks
df_JJ1 = getData(df_JJ1, 'jumpingjacks')
df_JJ2 = getData(df_JJ2, 'jumpingjacks')
df_JJ3 = getData(df_JJ3, 'jumpingjacks')
df_JJ4 = getData(df_JJ4, 'jumpingjacks')
df_JJ5 = getData(df_JJ5, 'jumpingjacks')

### Define Functions

In [ ]:
def loadDataFrames(name, pathAcc, pathGyr, pathOri):
    df_Acc= pd.read_csv(pathAcc)
    df_Gyr= pd.read_csv(pathGyr)
    df_Ori= pd.read_csv(pathOri)
    print("Data Frames for " + name + " loaded.")
    return df_Acc, df_Gyr, df_Ori

def loadJsonData(name, pathJson):
    df = pd.read_json(pathJson)
    print("Data Frame for " + name + " loaded.")
    return df	   

#Concat Funktion
def concatDataFrames(name, df_Acc, df_Gyr, df_Ori):
  df_Gyr.drop(['time','seconds_elapsed'], axis=1, inplace=True)
  df_Ori.drop(['time','seconds_elapsed'], axis=1, inplace=True)
  df = pd.concat([df_Acc, df_Gyr, df_Ori], axis=1)
  print("Data Frames for " + name + " concatenated.")
  return df

#Function to plot Gyr Data
def plotDataGyr(data, title):
    plt.plot(data['time'], data['z'], color='red', marker='.')
    plt.plot(data['time'], data['y'], color='blue', marker='.')
    plt.plot(data['time'], data['x'], color='green', marker='.')
    plt.xlabel('time')
    plt.ylabel('x,y,z')
    plt.legend(['z', 'y', 'x'])
    plt.title(title)
    plt.show()

#Function to Plot Acc Data
def plotDataAcc(data, title):
    plt.plot(data['time'], data['z'], color='red', marker='.')
    plt.plot(data['time'], data['y'], color='blue', marker='.')
    plt.plot(data['time'], data['x'], color='green', marker='.')
    plt.xlabel('time')
    plt.ylabel('x,y,z')
    plt.legend(['z', 'y', 'x'])
    plt.title(title)
    plt.show()

#Function to Plot Ori Data
def plotDataOri(data, title):
    plt.plot(data['time'], data['qz'], color='red', marker='.')
    plt.plot(data['time'], data['qy'], color='blue', marker='.')
    plt.plot(data['time'], data['qx'], color='green', marker='.')
    plt.plot(data['time'], data['qw'], color='yellow', marker='.')
    #plt.plot(data['time'], data['roll'], color='black', marker='.')
    plt.xlabel('time')
    plt.ylabel('qz,qy,qx,qw,roll')
    plt.legend(['qz', 'qy', 'qx', 'qw', 'roll'])
    plt.title(title)
    plt.show()

#Clean unused Sensors
def deleteSensorData(df):
  notUsedSensors = ['Annotation', 'Barometer', 'Battery', 'Brightness', 'Gravity', 'Light', 'Location', 'Magnetometer', 'MagnetometerUncalibrated', 'Microphone']
  df = df[~df['sensor'].isin(notUsedSensors)]
  return df


#Clean Function Json Data
def cleanDataJson(df):
    #Drop unused Columns
    columns_to_drop = ['sensors', 'version','device name','recording time','platform','appVersion', 'device id', 'sampleRateMs','relativeAltitude','pressure', 'batteryLevel', 'batteryState', 'lowPowerMode','brightness', 'lux', 'bearingAccuracy', 'speedAccuracy', 'verticalAccuracy', 'horizontalAccuracy', 'speed', 'bearing','altitude', 'longitude', 'latitude']
    columns_to_drop = list(set(columns_to_drop).intersection(df.columns))

    if columns_to_drop:
        df.drop(columns=columns_to_drop, inplace=True)
        print("Spalten wurden erfolgreich entfernt.")
    else:
        print("Keine der Spalten zum Entfernen gefunden.")
        #df.drop(['version','device name','recording time','platform','appVersion', 'device id', 'sampleRateMs' ], axis=1, inplace=True)
    return df


#Function to get Accelometer, Gyroscope and Orientation Data from Json in one Dataframe each
def getSensorData(df):
    df_Acc = df[df['sensor'] == 'Accelerometer']
    df_Gyr = df[df['sensor'] == 'Gyroscope']
    df_Ori = df[df['sensor'] == 'Orientation']
  #Drop all Columns with NaN Values
    df_Acc.dropna(axis=1, how='all', inplace=True)
    df_Gyr.dropna(axis=1, how='all', inplace=True)
    df_Ori.dropna(axis=1, how='all', inplace=True)

  #Drop sensor column
    df_Acc.drop(['sensor', 'seconds_elapsed'], axis=1, inplace=True)
    df_Gyr.drop(['sensor', 'seconds_elapsed'], axis=1, inplace=True)
    df_Ori.drop(['sensor', 'seconds_elapsed'], axis=1, inplace=True)
    

    return df_Acc, df_Gyr, df_Ori



### Load Data and give them a name

In [ ]:
#load mixed data
df_mixed_Acc, df_mixed_Gyr, df_mixed_Ori =loadDataFrames('mixed','data/MixedData/Accelerometer.csv', 'data/MixedData/Gyroscope.csv', 'data/MixedData/Orientation.csv')

##############################################################################################################

#load data frames for walking
df_walk1 = loadJsonData('walking 1', 'data/NormalWalk/NormalWalk.json')

#load data walk2
df_walk2 = loadJsonData('walking 2', 'data/Walk2/Walk2.json')

#load data walk3 Alex
df_walk3 = loadJsonData('walking 3', 'data\Alex_Rumstehen_-2023-05-02_15-30-58.json')

#load data walk4 Alex
df_walk4 = loadJsonData('walking 4', 'data\Alex_Rumstehen_2-2023-05-23_14-53-09.json')


##############################################################################################################

#load data frames for push ups
df_push1 = loadJsonData('push ups 1', 'data/PushUps/PushUps.json')

#load data pushups2
df_push2 = loadJsonData('push ups 2', 'data/PushUps2/PushUps2.json')

#load data pushups3 Alex
df_push3 = loadJsonData('push ups 3', 'data/Alex_Push_Up_2-2023-05-23_15-04-54.json')

#load data pushups4 Alex
df_push4 = loadJsonData('push ups 4', 'data\Alex_10_Liegestütz-2023-05-02_15-29-25.json')

#load data pushups5 Pierre
df_push5 = loadJsonData('push ups 5', 'data\PushUps-pierre.json')

##############################################################################################################

#load data frames for jumping jacks
df_JJ1 = loadJsonData('jumping jacks 1', 'data/JJ_rightHand/JJ1.json')

#load data JJ2
df_JJ2 = loadJsonData('jumping jacks 2', 'data/JJ2/JJ2.json')

#load data JJ3 Alex
df_JJ3 = loadJsonData('jumping jacks 3', 'data/Alex_10_Hampelmänner-2023-05-02_15-30-18.json')

#load data JJ4 Pierre
df_JJ4 = loadJsonData('jumping jacks 4', 'data/JumpingsJacks-pierre-1.json')

#load data JJ5 Pierre
df_JJ5 = loadJsonData('jumping jacks 5', 'data/JumpingsJacks-pierre-2.json')

In [ ]:
def df2_klassifizieren (
    df2: pd.DataFrame,
    sensor: str,
    schwellwert: float
) -> pd.DataFrame:
    """Klassifiziert die Daten des gegebenen Sensors in 'normal' und 'anomal'."""
    # Kopie des DataFrames erstellen
    df2_klassifiziert = df2.copy()
    # Spalte 'klassifikation' mit dem Wert 'normal' erstellen
    df2_klassifiziert['klassifikation'] = 'normal'
    # Alle Werte, die den Schwellwert überschreiten, als 'anomal' klassifizieren
    df2_klassifiziert.loc[df2_klassifiziert['value'] > schwellwert, 'klassifikation'] = 'anomal'
    # Klassifizierten DataFrame zurückgeben
    return df2_klassifiziert


In [ ]:
#iterieren über alldf mit der funktioin cleandatajson
for df in allDf:
    df = cleanDataJson(df)

In [ ]:
#Show Training Data
def showXTrainingData(data):
  print(data.shape)
  print(data.columns)
  plt.plot(data['z'], color='red', marker='.')#
  plt.plot(data['y'], color='blue', marker='.')#
  plt.plot(data['x'], color='green', marker='.')#
  plt.plot(data['qz'], color='yellow', marker='.')#
  plt.plot(data['qy'], color='black', marker='.')#
  plt.plot(data['qx'], color='orange', marker='.')#
  plt.plot(data['qw'], color='pink', marker='.')#
  plt.plot(data['roll'], color='purple', marker='.')#
  plt.xlabel('time')
  plt.ylabel('x,y,z,qz,qy,qx,qw,roll')
  plt.legend(['z', 'y', 'x', 'qz', 'qy', 'qx', 'qw', 'roll'])
  plt.title('Train Data')
  plt.show()

showXTrainingData(X_train)


In [ ]:
#Concat DataFrames
df = pd.concat([df_walk1, df_push1, df_JJ1], axis=0)
df_test = pd.concat([df_JJ2, df_push2, df_walk2], axis=0)

#Short Df_test for predciton
df_test= df_test.head(1517)



#Fill NaN Values with propagation
df.fillna(method='ffill', inplace=True)
df_test.fillna(method='ffill', inplace=True)



#kein random split sondern spezifizieren
#erste 70% train, letzte 30% test
#fünf trainings df und quasi im train nur die ersten 3 und die zwei anderen dann als test

In [ ]:
#Concat all Dataframes into one
df = pd.concat([df_walk1, df_walk2, df_push1, df_push2, df_push3, df_JJ1, df_JJ2, df_JJ3], axis=0)
df_test = pd.concat([df_walk3, df_walk4, df_push4, df_push5, df_JJ4, df_JJ5], axis=0)

In [ ]:
#Fill NaN Values with propagation
df.fillna(method='bfill', inplace=True)
df_test.fillna(method='bfill', inplace=True)

In [ ]:
df.head(10)

In [ ]:
print('Test: ', len(df_test))
print('Df: ', len(df))

df_test.drop(['label'], axis=1, inplace=True)

In [ ]:
#df_test.head(10)
df_test.describe()

# Sonstiges

In [ ]:
#take the uploaded file and load it into a dataframe and then apply the model to it
#df = pd.read_csv('uploaded_file.csv')
#model = pickle.load(open('knn.pkl', 'rb'))
#model.predict(df)


Lösung von ChatGpt, leider nicht so einfach...

In [ ]:
# Daten laden
training_data = df.drop(['label'], axis=1)
training_labels = df['label']

# Sensordaten in eine zweidimensionale Form umwandeln
training_data = training_data.reshape(training_data.shape[0], -1)


# KNN-Modell erstellen und trainieren
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn.fit(training_data, training_labels)

# Sensordaten für eine neue Instanz
new_data = df_test
new_data = new_data.reshape(1, -1)

# Vorhersage für die neue Instanz
prediction = knn.predict([new_data])

# Wahrscheinlichkeiten der Vorhersagen
probabilities = knn.predict_proba([new_data])

# Index der vorhergesagten Klasse
predicted_class_index = prediction[0]

# Wahrscheinlichkeit der vorhergesagten Klasse
predicted_class_probability = probabilities[0][predicted_class_index]

# Klassenbezeichnungen
class_names = ['Jumping Jacks', 'Pushups', 'Laufen']

# Ergebnis ausgeben
result = f"Mit {predicted_class_probability*100:.2f}% Sicherheit hat die Person {class_names[predicted_class_index]} gemacht."
print(result)
